<a href="https://colab.research.google.com/github/Homaoa/Classification-for-a-Bank-Marketing-Data-Set/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [47]:
url='/content/bank_marketing_cleaned_data.csv'
df=pd.read_csv(url)
df.head()

,Unnamed: 0,Age,Job,Marital,Education,Default,Housing,Loan,Contact,Month,...,Campaign,Pdays,Previous,Poutcome,Emp_var_rate,Cons_price_idx,Cons_conf_idx,Euribor3m,Nr_employed,Decision
0,0,56,"""housemaid""","""married""","""basic.4y""","""no""","""no""","""no""","""telephone""","""may""",...,1,999,0,"""nonexistent""",1.1,93.994,-36.4,4.857,5191.0,"""no"""
1,1,57,"""services""","""married""","""high.school""","""unknown""","""no""","""no""","""telephone""","""may""",...,1,999,0,"""nonexistent""",1.1,93.994,-36.4,4.857,5191.0,"""no"""
2,2,37,"""services""","""married""","""high.school""","""no""","""yes""","""no""","""telephone""","""may""",...,1,999,0,"""nonexistent""",1.1,93.994,-36.4,4.857,5191.0,"""no"""
3,3,40,"""admin.""","""married""","""basic.6y""","""no""","""no""","""no""","""telephone""","""may""",...,1,999,0,"""nonexistent""",1.1,93.994,-36.4,4.857,5191.0,"""no"""
4,4,56,"""services""","""married""","""high.school""","""no""","""no""","""yes""","""telephone""","""may""",...,1,999,0,"""nonexistent""",1.1,93.994,-36.4,4.857,5191.0,"""no"""


In [ ]:
#In EDA part, I have found the variables that are the best choices for developing the model

In [48]:
df['Decision']=df['Decision'].apply(lambda x:str(x).replace('"no"', '0'))
df['Decision']=df['Decision'].apply(lambda x:str(x).replace('"yes"', '1'))
df['Decision'].value_counts()

0    36547
1     4640
Name: Decision, dtype: int64

In [49]:
df_model=df[['Job','Marital','Education','Default','Contact','Month','Day','Pdays','Poutcome','Nr_employed', 'Euribor3m', 'Emp_var_rate', 'Previous', 'Cons_price_idx','Decision']]

In [ ]:
#Because I uploaded the data again in this new colab page, python used its own defualts for the type of the variables
#So I have to change the type of Pdays again. Also, for XGBoost model, I need to change the type of the target variable into numeric 0 and 1

In [50]:
df_model['Pdays']=df_model['Pdays'].astype('object')
df_model['Decision']=df_model['Decision'].astype('int')

<ipython-input-50-42aeb44be8a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Pdays']=df_model['Pdays'].astype('object')
<ipython-input-50-42aeb44be8a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Decision']=df_model['Decision'].astype('int')


In [51]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41187 entries, 0 to 41186
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Job             41187 non-null  object 
 1   Marital         41187 non-null  object 
 2   Education       41187 non-null  object 
 3   Default         41187 non-null  object 
 4   Contact         41187 non-null  object 
 5   Month           41187 non-null  object 
 6   Day             41187 non-null  object 
 7   Pdays           41187 non-null  object 
 8   Poutcome        41187 non-null  object 
 9   Nr_employed     41187 non-null  float64
 10  Euribor3m       41187 non-null  float64
 11  Emp_var_rate    41187 non-null  float64
 12  Previous        41187 non-null  int64  
 13  Cons_price_idx  41187 non-null  float64
 14  Decision        41187 non-null  int64  
dtypes: float64(4), int64(2), object(9)
memory usage: 4.7+ MB


In [52]:
X=df_model[['Job','Marital','Education','Default','Contact','Month','Day','Pdays','Poutcome','Nr_employed', 'Euribor3m', 'Emp_var_rate', 'Previous', 'Cons_price_idx']]
y=df_model[['Decision']]

In [ ]:
#Now the categorical variables should be dummies

In [53]:
X = pd.get_dummies(X)

<ipython-input-53-a36baeb3b275>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  X = pd.get_dummies(X)


In [54]:
X.head()

,Nr_employed,Euribor3m,Emp_var_rate,Previous,Cons_price_idx,"Job_""admin.""","Job_""blue-collar""","Job_""entrepreneur""","Job_""housemaid""","Job_""management""",...,Pdays_20,Pdays_21,Pdays_22,Pdays_25,Pdays_26,Pdays_27,Pdays_999,"Poutcome_""failure""","Poutcome_""nonexistent""","Poutcome_""success"""
0,5191.0,4.857,1.1,0,93.994,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
1,5191.0,4.857,1.1,0,93.994,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,5191.0,4.857,1.1,0,93.994,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,5191.0,4.857,1.1,0,93.994,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,5191.0,4.857,1.1,0,93.994,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [ ]:
#Now I split the data into train and test

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
#After spliting, I scale the numeric variables which are the first 5 variables
#I fit the scaling with train data, and for test data I only transform

In [56]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[['Nr_employed','Euribor3m','Emp_var_rate','Previous','Cons_price_idx']]=sc.fit_transform(X_train[['Nr_employed','Euribor3m','Emp_var_rate','Previous','Cons_price_idx']])
X_test[['Nr_employed','Euribor3m','Emp_var_rate','Previous','Cons_price_idx']]=sc.transform(X_test[['Nr_employed','Euribor3m','Emp_var_rate','Previous','Cons_price_idx']])

In [ ]:
### Logistic regression model

In [57]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [58]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[9026  120]
 [ 897  254]]


0.9012333689424105

In [ ]:
### KNN model

In [59]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [60]:
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[8846  300]
 [ 832  319]]


0.890065067495387

In [ ]:
### SVM model

In [61]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear', random_state=0)

In [62]:
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[9038  108]
 [ 927  224]]


0.8994852869767893

In [ ]:
### Kernel SVM model

In [63]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(random_state=0)

In [64]:
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[9011  135]
 [ 851  300]]


0.904243954549869

In [ ]:
### Naive bayes model

In [65]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [66]:
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[8808  338]
 [ 786  365]]


0.8908419928134408

In [ ]:
### Decision tree classificatio model

In [67]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [68]:
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[8721  425]
 [ 805  346]]


0.8805477323492279

In [ ]:
### Random forest classification model

In [69]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

<ipython-input-69-85558699667f>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [70]:
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[8834  312]
 [ 821  330]]


0.8899679518306303

In [ ]:
### XGBoost model

In [71]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [72]:
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[8942  204]
 [ 822  329]]


0.9003593279595998

In [ ]:
###### Result : as we can see, Kernel SVM model has the best performance with accuracy score = 0.904243954549869
######          after that, it is Logistic regression model with accuracy score = 0.9012333689424105